In [2]:
import pandas as pd
from datetime import datetime

In [3]:
rating = pd.read_csv('ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [74]:
user_id_max = (rating.groupby('userId').
               max().reset_index().
               rename(columns = {'timestamp':'timestamp_max'})
              )
# user_id_max['max'] = user_id_max.apply(lambda row: datetime.utcfromtimestamp(row['timestamp_max']).date(), axis = 1)
user_id_min = rating.groupby('userId').min().reset_index().rename(columns = {'timestamp':'timestamp_min'})
user_id_100 = rating.groupby('userId').count().reset_index()[['userId', 'rating']]
user_id_100['true'] = user_id_100.apply(lambda row: 'Больше 100 оценок' if row['rating'] > 100 else None, axis = 1)
user_id_100_only = user_id_100[user_id_100['true'] == 'Больше 100 оценок']
result = user_id_100_only.merge(user_id_max, on = 'userId', how = 'left').merge(user_id_min, on = 'userId', how = 'left')
# result['delta'] = (
#     result.apply
#     (lambda row: datetime.utcfromtimestamp(row['timestamp_max'])
#      - datetime.utcfromtimestamp(row['timestamp_min']), axis = 1)
# )
result['delta'] = (
    result.apply
    (lambda row: (row['timestamp_max']
     - row['timestamp_min']), axis = 1)
    
)
result.head()[['userId', 'delta']] 
# itog = result.groupby('true').mean().reset_index()[['true','delta']]
# itog.head()
    

,userId,delta
0,4,203560
1,8,85187
2,15,471393496
3,17,8053
4,19,5282


In [10]:
user_id = rating.groupby('userId').agg({'timestamp': ['min', 'max'], 'userId': 'count'}).reset_index()
user_id.head()

userId   timestamp             userId
                 min         max  count
0      1  1260759108  1260759205     20
1      2   835355395   835356246     76
2      3  1298861589  1298932787     51
3      4   949778714   949982274    204
4      5  1163373044  1163375145    100

In [11]:
user_id = user_id[user_id['userId']['count'] > 100]
user_id.head()

userId   timestamp             userId
                  min         max  count
3       4   949778714   949982274    204
7       8  1154389340  1154474527    116
14     15   997937239  1469330735   1700
16     17  1127468587  1127476640    363
18     19   855190091   855195373    423

In [13]:
user_id.loc[:,'livetime'] = user_id['timestamp']['max'] - user_id['timestamp']['min']
user_id.head()

userId     timestamp                userId     livetime
                    min           max   count             
3     4.0  9.497787e+08  9.499823e+08   204.0     203560.0
7     8.0  1.154389e+09  1.154475e+09   116.0      85187.0
14   15.0  9.979372e+08  1.469331e+09  1700.0  471393496.0
16   17.0  1.127469e+09  1.127477e+09   363.0       8053.0
18   19.0  8.551901e+08  8.551954e+08   423.0       5282.0

In [14]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [15]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [16]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [17]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [32]:
revenue_with_adress = (client_base.merge(rzd, on = 'client_id', how = 'left').
           merge(auto, on = 'client_id', how = 'left').
           merge(air, on = 'client_id', how = 'left').
           fillna(0)            
          )
revenue_with_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


In [33]:
revenue = revenue_with_adress.drop('address', axis = 1)
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
